Data Validation of the top 10 imbalanced route: \
(analysis_id = 1453262, 1453284, 1453305, 1453367, 1453395, 1453445, 1453464, 1453483, 1454196, 1454209, 1454352, 1454366, 1454523, 1454549, 1454670, 1454683, 1455243, 1455256, 1455385, 1455400) \
NOTE: No bt data found for 1455243 , 1455400 \
Period: 2 days (From '2019-10-09 00:00:00' to '2019-10-10 23:59:00')

In [42]:
%matplotlib inline
from psycopg2 import connect
import psycopg2.sql as pg
import configparser
from datetime import datetime, timedelta, date
from itertools import cycle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#import matplotlib.cm as cm
#from matplotlib.ticker import FuncFormatter
import pandas as pd
import pandas.io.sql as pandasql
import seaborn as sns
from IPython.display import HTML
def print_table(sql, con):
    return HTML(pandasql.read_sql(sql, con).to_html(index=False))

In [43]:
# setting up pgsql connection
CONFIG = configparser.ConfigParser()
CONFIG.read(r'/home/jchew/local/db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [44]:
# Select bt observations
sql = '''SELECT * FROM jchew.bt_top10_2days'''
print_table(sql, con)

analysis_id,bt_id,street_name,direction,from_intersection,to_intersection,sum_bt_obs
1453305,8,Dundas,EB,Spadina,University,306
1453445,15,Dundas,WB,University,Spadina,178
1454196,53,King,EB,Spadina,University,11
1453367,11,Dundas,EB,Jarvis,Parliament,593
1455385,117,University,NB,Queen,Dundas,298
1454352,61,King,WB,Yonge,University,34
1454670,77,Front,WB,University,Spadina,43
1453284,7,Dundas,EB,Bathurst,Spadina,475
1454209,54,King,EB,University,Yonge,40
1454549,71,Front,EB,Spadina,University,87


In [45]:
# Finding ratio for bt observations
sql = '''
WITH X AS (
SELECT a.analysis_id AS analysis_id_1, a.bt_id AS bt_id_1, 
	b.analysis_id AS analysis_id_2, b.bt_id AS bt_id_2, a.street_name, 
	a.direction AS eb_nb, b.direction AS wb_sb, 
	a.from_intersection AS intersection_1, a.to_intersection AS intersection_2,
	a.sum_bt_obs AS eb_nb_obs, b.sum_bt_obs AS wb_sb_obs
	FROM jchew.bt_top10_2days a
JOIN jchew.bt_top10_2days b
ON a.street_name = b .street_name AND a.from_intersection = b.to_intersection 
AND a.to_intersection=b.from_intersection 
WHERE a.direction IN ('EB', 'NB')
)
SELECT *,
CASE WHEN X.eb_nb_obs > X.wb_sb_obs THEN (X.eb_nb_obs * 1.0) / (X.eb_nb_obs + X.wb_sb_obs)
WHEN X.eb_nb_obs < X.wb_sb_obs THEN (X.wb_sb_obs * 1.0) / (X.eb_nb_obs + X.wb_sb_obs)
END AS "EB/WB or NB/SB Ratio",
 CASE
WHEN X.eb_nb = 'EB' THEN CASE
	WHEN X.eb_nb_obs < X.wb_sb_obs THEN 'WB'
    WHEN X.eb_nb_obs > X.wb_sb_obs THEN 'EB'
    END
WHEN X.eb_nb = 'NB' THEN CASE
    WHEN X.eb_nb_obs < X.wb_sb_obs THEN 'SB'
    WHEN X.eb_nb_obs > X.wb_sb_obs THEN 'NB'
    END
        END AS "Bias Towards"
FROM X
ORDER BY analysis_id_1
'''
print_table(sql, con)

analysis_id_1,bt_id_1,analysis_id_2,bt_id_2,street_name,eb_nb,wb_sb,intersection_1,intersection_2,eb_nb_obs,wb_sb_obs,EB/WB or NB/SB Ratio,Bias Towards
1453262,6,1453483,17,Dundas,EB,WB,Dufferin,Bathurst,310,265,0.539130,EB
1453284,7,1453464,16,Dundas,EB,WB,Bathurst,Spadina,475,371,0.561466,EB
1453305,8,1453445,15,Dundas,EB,WB,Spadina,University,306,178,0.632231,EB
1453367,11,1453395,12,Dundas,EB,WB,Jarvis,Parliament,593,303,0.661830,EB
1454196,53,1454366,62,King,EB,WB,Spadina,University,11,21,0.656250,WB
1454209,54,1454352,61,King,EB,WB,University,Yonge,40,34,0.540541,EB
1454523,70,1454683,78,Front,EB,WB,Bathurst,Spadina,297,241,0.552045,EB
1454549,71,1454670,77,Front,EB,WB,Spadina,University,87,43,0.669231,EB
1455385,117,1455256,110,University,NB,SB,Queen,Dundas,298,829,0.735581,SB


In [46]:
# To compare ratios for both bt and here
sql = '''
SELECT b.analysis_id_1, b.analysis_id_2, b.street_name, 
b.eb_nb, b.wb_sb, b.intersection_1, b.intersection_2, 
a.eb_nb_obs AS bt_eb_nb_obs, a.wb_sb_obs AS bt_wb_sb_obs,
b.eb_nb_obs AS here_eb_nb_obs, b.wb_sb_obs AS here_wb_sb_obs,
a."EB/WB or NB/SB Ratio" AS bt_ratio,
a."Bias Towards" AS bt_bias,
b."EB/WB or NB/SB Ratio" AS here_ratio,
b."Bias Towards" AS here_bias
FROM jchew.ratio_bt_top10_2days a
RIGHT JOIN jchew.ratio_here_top10_2days b
USING (analysis_id_1)
'''
print_table(sql, con)

analysis_id_1,analysis_id_2,street_name,eb_nb,wb_sb,intersection_1,intersection_2,bt_eb_nb_obs,bt_wb_sb_obs,here_eb_nb_obs,here_wb_sb_obs,bt_ratio,bt_bias,here_ratio,here_bias
1453262,1453483,Dundas,EB,WB,Dufferin,Bathurst,310.0,265.0,4059.0,4138.0,0.539130,EB,0.504819,WB
1453284,1453464,Dundas,EB,WB,Bathurst,Spadina,475.0,371.0,1842.0,2012.0,0.561466,EB,0.522055,WB
1453305,1453445,Dundas,EB,WB,Spadina,University,306.0,178.0,2082.0,1942.0,0.632231,EB,0.517396,EB
1453367,1453395,Dundas,EB,WB,Jarvis,Parliament,593.0,303.0,4055.0,1899.0,0.661830,EB,0.681055,EB
1454196,1454366,King,EB,WB,Spadina,University,11.0,21.0,312.0,296.0,0.656250,WB,0.513158,EB
1454209,1454352,King,EB,WB,University,Yonge,40.0,34.0,687.0,324.0,0.540541,EB,0.679525,EB
1454523,1454683,Front,EB,WB,Bathurst,Spadina,297.0,241.0,528.0,672.0,0.552045,EB,0.560000,WB
1454549,1454670,Front,EB,WB,Spadina,University,87.0,43.0,1931.0,1345.0,0.669231,EB,0.589438,EB
1455385,1455256,University,NB,SB,Queen,Dundas,298.0,829.0,1106.0,1361.0,0.735581,SB,0.551682,SB
1455400,1455243,University,NB,SB,Dundas,College,NaN,NaN,3158.0,4806.0,NaN,None,0.603466,SB
